In [1]:
from cassandra.cluster import Cluster
import station_pb2_grpc
import station_pb2
import pandas as pd
import sys
import os
import subprocess
import grpc
from datetime import date

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType,StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import corr
from pyspark.sql.functions import col
import matplotlib.pyplot as plt

In [2]:
cluster = Cluster(['p6_db_2', 'p6_db_1', 'p6_db_3'])
session = cluster.connect()

In [3]:
spark = (SparkSession.builder
         .appName("cassandra-connector")
         .config("spark.driver.allowMultipleContexts", "true")
         .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.2.0")
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .config("spark.cassandra.connection.host", "p6_db_3,p6_db_2,p6_db_1")  # Cassandra nodes
         .config("spark.cassandra.connection.port", "9042")  # Cassandra port
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-eeab7d1a-4622-4501-8ad5-af170f83e7cb;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.2.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.2.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central


	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.github.spotbugs#spotbugs-annotations;3.1.12 in central
	found com.google.code.findbugs#jsr305;3.0.2 in central


	found com.datastax.oss#java-driver-mapper-runtime;4.13.0 in central
	found com.datastax.oss#java-driver-query-builder;4.13.0 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.scala-lang#scala-reflect;2.12.11 in central
:: resolution report :: resolve 564ms :: artifacts dl 21ms
	:: modules in use:
	com.datastax.oss#java-driver-core-shaded;4.13.0 from central in [default]
	com.datastax.oss#java-driver-mapper-runtime;4.13.0 from central in [default]
	com.datastax.oss#java-driver-query-builder;4.13.0 from central in [default]
	com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 from central in [default]
	com.datastax.oss#native-protocol;1.5.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector-driver_2.12;3.2.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector_2.12;3.2.0 from central in [default]
	com.github.spotbugs#spotbugs-annotations;3.1.12 from c

23/09/04 04:00:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Part 1

In [10]:
def setup_cassandra_table():
    # TODO: Q1
    raise NotImplementedError()

In [11]:
# Q1 Ans
setup_cassandra_table()
print(session.execute("describe keyspace weather").one().create_statement)
print(session.execute("describe type weather.station_record").one().create_statement)
print(session.execute("describe table weather.stations").one().create_statement)

CREATE KEYSPACE weather WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}  AND durable_writes = true;
CREATE TYPE weather.station_record (
    tmin int,
    tmax int
);
CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    record frozen<station_record>,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_

### Adding metadata to table

In [12]:
table = spark.read.option("header", True).csv("/datasets/stations_metadata.csv")
table.printSchema()

root
 |-- ID: string (nullable = true)
 |-- LATITUDE: string (nullable = true)
 |-- LONGITUDE: string (nullable = true)
 |-- ELEVATION: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- GSN FLAG: string (nullable = true)
 |-- HCN/CRN FLAG: string (nullable = true)
 |-- WMO ID: string (nullable = true)



In [13]:
# TODO: Code to insert metadata into weather.stations

In [14]:
# Metadata insert verify
result = session.execute("SELECT COUNT(*) FROM weather.stations")
print(result.one()[0])

1313


In [15]:
def get_tokens(station_id):
    # TODO: Q2
    raise NotImplementedError() 

In [16]:
# Q2 Ans
row_token, vnode_token = get_tokens("USC00470273")
print("Row token:", row_token)
print("Vnode token:", vnode_token)

Row token: -1
Vnode token: -1


## Part 2

In [ ]:
# TODO: Connect to the server
channel = None
stub = None

In [ ]:
def simulate_sensor(sensor_id):
    # TODO: gRPC client simulation
    raise NotImplementedError()

In [ ]:
# gRPC client runner
for station in ["USW00014837", "USR0000WDDG", "USW00014898", "USW00014839"]:
    num_failed = simulate_sensor(station)
    if num_failed > 0:
        print("Failed to write all data for", station)
        break
    
    r = stub.StationMax(station_pb2.StationMaxRequest(station=station))
    if r.error:
        print(f"error {r.error} in getting max temp of {station}")
    else:
        print(f"max temp for {station} is {r.tmax}")

## Part 3

In [4]:
cassandra_df = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="stations", keyspace="weather") \
    .load()

In [5]:
def create_weather_view():
    # TODO: Initialize the weather2022 view
    raise NotImplementedError()

In [6]:
# Weather2022 verify
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [7]:
# TODO: Q3


In [8]:
# TODO: Q4

## Part 4

In [9]:
# TODO: Q5


In [10]:
# TODO: Q6

In [11]:
# TODO: Q7

In [12]:
spark.stop()
session.shutdown()
cluster.shutdown()